# Battle Of The Neighbourhoods

## Introduction: Business Problem
In this project, I aim to find the similarity between the placement of hospitals in the central NYC, New York and Bangkok, Thailand.

## Data

In [1]:
from geopy.geocoders import Nominatim
import requests

In [2]:
CLIENT_ID = "3UBZY5XRBMZOKIVFEYCHGMKTUA5ITDYTNLENPXFU5BEDDBRX"
CLIENT_SECRET = "J1FK2RDAPCCX0MNLCFPLLYE3T1IL0OHCRGY3AJ3R1NUWJ4IJ"

In [3]:
def get_coord(address):
    geolocator = Nominatim(user_agent="Test")
    location = geolocator.geocode(address)
    return [location.latitude, location.longitude]

bangkok_location = get_coord("Bangkok, Thailand")
nyc_location = get_coord("New York, New York")
print(bangkok_location)
print(nyc_location)

[13.7542529, 100.493087]
[40.7127281, -74.0060152]


In [4]:
LIMIT = 130
DISTANCE = 1000
cat = "4bf58dd8d48988d104941735"

bangkok_url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v=20180605&ll={bangkok_location[0]},{bangkok_location[1]}&radius={DISTANCE}&limit={LIMIT}&categoryId={cat}"
nyc_url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v=20180605&ll={nyc_location[0]},{nyc_location[1]}&radius={DISTANCE}&limit={LIMIT}&query=hospital&categoryId={cat}"

In [5]:
bangkok_hospitals = requests.get(bangkok_url).json()
bangkok_hospitals

{'meta': {'code': 200, 'requestId': '5f193398802bda45ac96884e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Phra Borom Maha Ratchawang',
  'headerFullLocation': 'Phra Borom Maha Ratchawang, Bangkok',
  'headerLocationGranularity': 'neighborhood',
  'query': 'medical',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 13.763252909000007,
    'lng': 100.50233540838725},
   'sw': {'lat': 13.745252890999991, 'lng': 100.48383859161275}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc4d8a974a9a5939070d6f6',
       'name': '100th Anniversary Siriraj Pavilion (ศาลาศิริราช 100 ปี)',
       'location': {'address': 'Siriraj Hospital',
        'lat': 13.757213031906385,
     

In [6]:
nyc_hospitals = requests.get(nyc_url).json()
nyc_hospitals

{'meta': {'code': 200, 'requestId': '5f1935cf2321bc05cd7dee21'},
 'response': {'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hospital',
  'totalResults': 190,
  'suggestedBounds': {'ne': {'lat': 40.72172810900001,
    'lng': -73.99416384126839},
   'sw': {'lat': 40.70372809099999, 'lng': -74.0178665587316}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a75e7aef964a52098e11fe3',
       'name': 'Water4Dogs Rehabilitaion Center',
       'location': {'address': '77 Worth St',
        'crossStreet': 'Broadway',
        'lat': 40.71683833242527,
        'lng': -74.00592175519765,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.71683833242527,
   

In [7]:
def get_category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
import pandas as pd
from pandas.io.json import json_normalize

In [26]:
bangkok_venues = bangkok_hospitals['response']['groups'][0]['items']
bangkok_nearby_venues = json_normalize(bangkok_venues)

bangkok_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
bangkok_nearby_venues = bangkok_nearby_venues.loc[:, bangkok_filtered_columns]

bangkok_nearby_venues['venue.categories'] = bangkok_nearby_venues.apply(get_category, axis=1)
bangkok_nearby_venues.columns = [col.split(".")[-1] for col in bangkok_nearby_venues.columns]
bangkok_nearby_venues = bangkok_nearby_venues[bangkok_nearby_venues['categories']!="Massage Studio"]
bangkok_nearby_venues = bangkok_nearby_venues[bangkok_nearby_venues['categories']!="Medical School"]

bangkok_nearby_venues.head()

/home/rcyt17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,100th Anniversary Siriraj Pavilion (ศาลาศิริรา...,Medical Center,13.757213,100.485451
1,Poonthavee Drugstore (พูลทวีเภสัช),Pharmacy,13.756124,100.484535
3,ตึกพระศรีสวรินทรา,Medical Center,13.758969,100.488386
4,Srisangwan Building (ตึกศรีสังวาลย์),Medical Center,13.757492,100.486306
5,SIRIRAJ ACUTE STROKE UNIT,Medical Center,13.757247,100.484996


In [27]:
bangkok_hospital_counts = bangkok_nearby_venues.shape[0]
print(f"Count: {bangkok_nearby_venues.shape[0]}")
print(set(bangkok_nearby_venues['categories']))

Count: 44
{'Alternative Healer', 'Mental Health Office', "Doctor's Office", 'Medical Center', 'Pharmacy', 'Hospital', 'Building'}


In [28]:
nyc_venues = nyc_hospitals['response']['groups'][0]['items']
nyc_nearby_venues = json_normalize(nyc_venues)

nyc_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nyc_nearby_venues = nyc_nearby_venues.loc[:, nyc_filtered_columns]

nyc_nearby_venues['venue.categories'] = nyc_nearby_venues.apply(get_category, axis=1)
nyc_nearby_venues.columns = [col.split(".")[-1] for col in nyc_nearby_venues.columns]
nyc_nearby_venues = nyc_nearby_venues[nyc_nearby_venues['categories'] != 'Chiropractor']

nyc_nearby_venues.head()

/home/rcyt17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Water4Dogs Rehabilitaion Center,Medical Center,40.716838,-74.005922
1,Weill Cornell Medical Associates,Medical Center,40.716684,-74.006724
2,CP Advanced Imaging,Medical Center,40.716596,-73.996254
3,AdvantageCare Physicians,Medical Center,40.714264,-74.004013
4,Foot Doctor of NY,Medical Center,40.714907,-74.005844


In [29]:
nyc_hospital_counts = nyc_nearby_venues.shape[0]
print(f"Count: {nyc_nearby_venues.shape[0]}")
print(set(nyc_nearby_venues['categories']))

Count: 99
{'Mental Health Office', "Doctor's Office", 'Assisted Living', 'Medical Center', 'Pharmacy', 'Hospital', 'Eye Doctor'}
